In [5]:
# 連接雲端硬碟(測試資料存在雲端)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 建立residual layer的class
class basicBlock(keras.layers.Layer):

    # kernel_size卷積層大小預設為3，stride為1，filter_num代表輸出多少層filter
    # 如果經過卷積層的filter_num跟原始輸入的filter_num不一樣，下面self.downsample.add會error。
    # 將change = True讓filter_num保持一致
    def __init__(self, filter_num, stride=1, change=False, kernel_size=3):
        super(basicBlock, self).__init__()

        # 內含兩個卷積層
        # padding='same'讓輸出維持19*19*X
        # 卷積層1
        self.conv1 = keras.layers.Conv2D(filter_num, kernel_size=kernel_size, strides=stride, padding='same')

        self.bn1 = keras.layers.BatchNormalization()
        self.relu = keras.layers.Activation('relu')

        # 卷積層2
        self.conv2 = keras.layers.Conv2D(filter_num, kernel_size=kernel_size, strides=stride, padding='same')
        self.bn2 = keras.layers.BatchNormalization()

        # 未經過卷積層的input
        if change != False:
            self.downsample = keras.Sequential()
            self.downsample.add(keras.layers.Conv2D(filter_num, kernel_size=1, strides=stride, padding='same'))
        else:
            self.downsample = lambda x: x

    def call(self, inputs, training=None):

        #前向計算forward
        out = self.conv1(inputs)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        #inputs通過identity轉換
        identity = self.downsample(inputs)

        # f(x)+x運算
        output = keras.layers.add([out, identity])

        # 再通過relu激活函數並回傳
        output = tf.nn.relu(output)
        return output

# 多個residual layer組成的block
def build_resblock(filter_num, blocks, stride=1, change=False, kernel=3):
    Resblock = keras.Sequential()
    Resblock.add(basicBlock(filter_num=filter_num, stride=stride, change=change, kernel_size=kernel))

    #第一個之後的residual layer的stride固定是1
    for i in range(1, blocks):
        Resblock.add(basicBlock(filter_num=filter_num, stride=1, change=change, kernel_size=kernel))
    return Resblock

# 建模型
def create_model():
    inputs = keras.layers.Input(shape=(19, 19, 8))
    outputs = keras.layers.Conv2D(kernel_size=3, filters=64, strides=1, padding='same')(inputs)
    outputs = keras.layers.BatchNormalization()(outputs)
    outputs = keras.layers.Activation('relu')(outputs)

    outputs = build_resblock(filter_num=64, blocks=2, kernel=3)(outputs)
    outputs = build_resblock(filter_num=64, blocks=2, stride=1, change=True, kernel=3)(outputs)
    outputs = build_resblock(filter_num=128, blocks=2, stride=1, change=True, kernel=3)(outputs)
    outputs = keras.layers.AveragePooling2D(pool_size=(2,2))(outputs)
    outputs = build_resblock(filter_num=128, blocks=2, stride=1, change=True, kernel=3)(outputs)
    outputs = keras.layers.AveragePooling2D(pool_size=(2,2))(outputs)

    # 全連接層
    outputs = keras.layers.Flatten()(outputs)
    outputs = keras.layers.BatchNormalization()(outputs)
    outputs = keras.layers.Dense(256, activation='relu')(outputs)
    outputs = keras.layers.Dense(3, activation='softmax')(outputs)

    model = tf.keras.models.Model(inputs, outputs)

    return model

In [6]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}

def search(all, target, row, column, visited, live):
    visited.add((row, column))
    directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]  # Up, down, left, right
    for dr, dc in directions:
        r, c = row + dr, column + dc
        if 0 <= r <= 18 and 0 <= c <= 18 and (r, c) not in visited:
            if all[r][c] == 0 and target[r][c] == 0:
                return True
            elif target[r][c] == 1:
                live = search(all, target, r, c, visited, live)
                if live:
                    return live
    return False

def dieCheck(all, target, row, column):
    num = 4
    directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]  # Up, down, left, right
    for dr, dc in directions:
        r, c = row + dr, column + dc
        if r < 0 or r > 18 or c < 0 or c > 18 or all[r][c] == 1:
            num -= 1
    if num == 0:
        target[row][column] = 0
        return target
    else:
        num -= sum(target[row + dr][column + dc] == 1 for dr, dc in directions if 0 <= row + dr <= 18 and 0 <= column + dc <= 18)
        if num == 0:
            live = False
            visited = set()
            for dr, dc in directions:
                r, c = row + dr, column + dc
                if 0 <= r <= 18 and 0 <= c <= 18 and target[r][c] == 1 and (r, c) not in visited:
                    live = search(all, target, r, c, visited, live)
                    if live:
                        break
            if not live:
                for r, c in visited:
                    target[r][c] = 0
    return target

def twoColor(moves, previous_board=None):
    if previous_board is None:
        blackAll = np.zeros((19,19))
        whiteAll = np.zeros((19,19))
    else:
        blackAll, whiteAll = previous_board
        moves = [moves[-1]]

    for i in moves:
        column = int(i[2:4])
        row = int(i[4:6])
        color = i[0]
        if color == 'B':
            blackAll[row][column] = 1
            target = whiteAll
            all = blackAll
        else:
            whiteAll[row][column] = 1
            target = blackAll
            all = whiteAll

        arr = [0, 0, 0, 0]
        if row > 0 and target[row-1][column] == 1:
            arr[0] = 1
        if row < 18 and target[row+1][column] == 1:
            arr[1] = 1
        if column > 0 and target[row][column-1] == 1:
            arr[2] = 1
        if column < 18 and target[row][column+1] == 1:
            arr[3] = 1

        for i in range(4):
            if arr[i]==1:
                if i==0:
                    target = dieCheck(all, target=target, row=row-1, column=column)
                elif i==1:
                    target = dieCheck(all, target=target, row=row+1, column=column)
                elif i==2:
                    target = dieCheck(all, target=target, row=row, column=column-1)
                elif i==3:
                    target = dieCheck(all, target=target, row=row, column=column+1)

        if color == 'B':
            whiteAll = target
        else:
            blackAll = target

    return blackAll, whiteAll

def liberties(board):
    liberties_board = np.zeros((19,19))
    directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]  # Up, down, left, right
    visited = set()
    liberties_visited = set()

    for row in range(19):
        for column in range(19):
            if board[row][column] != 0 and (row, column) not in visited:  # If there is a stone at this position
                group = [(row, column)]
                liberties_count = 0
                liberties_visited.clear()
                visited.add((row, column))

                # Find all stones in the same group
                i = 0
                while i < len(group):
                    r, c = group[i]
                    for dr, dc in directions:
                        nr, nc = r + dr, c + dc
                        if 0 <= nr <= 18 and 0 <= nc <= 18:
                            if board[nr][nc] == 0 and (nr, nc) not in liberties_visited:  # If the adjacent position is empty
                                liberties_count += 1
                                liberties_visited.add((nr, nc))
                            elif board[nr][nc] == board[row][column] and (nr, nc) not in visited:  # If the adjacent stone is the same color
                                group.append((nr, nc))
                                visited.add((nr, nc))
                    i += 1

                # Assign the liberties count to all stones in the same group
                for r, c in group:
                    liberties_board[r][c] = liberties_count
    return liberties_board

def prepare_input_for_dan_kyu_models(moves):
    array = np.zeros((19,19,8))

    target_step = len(moves)
    # 前兩步
    if target_step > 2:
        array[:,:,0], array[:,:,1] = twoColor(moves[:target_step - 2])
    # 前一步
    if target_step > 1:
        array[:,:,2], array[:,:,3] = twoColor([moves[target_step - 2]], previous_board=(array[:,:,0], array[:,:,1]))
    # 目前這一步
    array[:,:,4], array[:,:,5] = twoColor([moves[target_step - 1]], previous_board=(array[:,:,2], array[:,:,3]))

    # 計算氣
    array[:,:,6] = liberties(array[:,:,4] + array[:,:,5] * 2) / 10

    target_col = int(moves[-1][2:4])
    target_row = int(moves[-1][4:6])

    # 單獨最後一步
    array[target_row, target_col,7] = 1

    return array

def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

# 清除play_style_test_private中空白的資料
def not_empty(s):
    return s and s.strip()

model = create_model()
model.load_weights('/content/best.h5')

df = open('/content/drive/MyDrive/play_style_test_private.csv').read().splitlines()
games_id = [i.split(',',1)[0] for i in df]
games = [i.split(',',1)[-1] for i in df]

x_testing = []

i=1

for game in games:
    for char in chars:
        if (len(str(coordinates[char])) == 1):
            num = '0' + str(coordinates[char])
        else:
            num = str(coordinates[char])
        game = game.replace(char, num)
    moves_list = game.split(',')
    moves_list = list(filter(not_empty,moves_list))
    x_testing.append(prepare_input_for_dan_kyu_models(moves_list))

x_testing =np.array(x_testing)

print('shape: '+str(x_testing.shape))

prediction = model.predict(x_testing)
prediction_numbers = np.argmax(prediction, axis=1)

with open('/content/drive/MyDrive/play_style_answer_private.csv', 'a') as f:
    for index, number in enumerate(prediction_numbers):
        answer_row = games_id[index] + ',' + str(number+1) + '\n'
        f.write(answer_row)

shape: (9998, 19, 19, 8)
313/313 [==============================] - 163s 518ms/step
